In [1]:
# -*- coding: utf-8 -*-

import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from collections import namedtuple



In [2]:

def to_array(x):
    """ Convert an vector to array if needed """
    if len(x.shape)==1:
        x=x.reshape(1,x.shape[0])
    return x


def gen_arti(centerx=1,centery=1,sigma=0.1,nbex=1000,data_type=0,epsilon=0.02):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param epsilon: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
        #melange de 2 gaussiennes
        xpos=np.random.multivariate_normal([centerx,centerx],np.diag([sigma,sigma]),nbex/2)
        xneg=np.random.multivariate_normal([-centerx,-centerx],np.diag([sigma,sigma]),nbex/2)
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex/2),-np.ones(nbex/2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centerx],np.diag([sigma,sigma]),nbex/4),np.random.multivariate_normal([-centerx,-centerx],np.diag([sigma,sigma]),nbex/4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centerx],np.diag([sigma,sigma]),nbex/4),np.random.multivariate_normal([centerx,-centerx],np.diag([sigma,sigma]),nbex/4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex/2),-np.ones(nbex/2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,epsilon,nbex)
    data[:,1]+=np.random.normal(0,epsilon,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

def plot_data(data,labels=None):
    """
    Affiche des donnees 2D
    :param data: matrice des donnees 2d
    :param labels: vecteur des labels (discrets)
    :return:
    """
    cols,marks = ["red", "green", "blue", "orange", "black", "cyan"],[".","+","*","o","x","^"]
    if labels is None:
        plt.scatter(data[:,0],data[:,1],marker="x")
        return
    for i,l in enumerate(sorted(list(set(labels.flatten())))):
        plt.scatter(data[labels==l,0],data[labels==l,1],c=cols[i],marker=marks[i])



def make_grid(data=None,xmin=-5,xmax=5,ymin=-5,ymax=5,step=20):
    """ Cree une grille sous forme de matrice 2d de la liste des points
    :param data: pour calcluler les bornes du graphe
    :param xmin: si pas data, alors bornes du graphe
    :param xmax:
    :param ymin:
    :param ymax:
    :param step: pas de la grille
    :return: une matrice 2d contenant les points de la grille
    """
    if data!=None:
        xmax, xmin, ymax, ymin = np.max(data[:,0]),  np.min(data[:,0]), np.max(data[:,1]), np.min(data[:,1])
    x, y =np.meshgrid(np.arange(xmin,xmax,(xmax-xmin)*1./step), np.arange(ymin,ymax,(ymax-ymin)*1./step))
    grid=np.c_[x.ravel(),y.ravel()]
    return grid, x, y


def plot_frontiere(data,f,step=20):
    """ Trace un graphe de la frontiere de decision de f
    :param data: donnees
    :param f: fonction de decision
    :param step: pas de la grille
    :return:
    """
    grid,x,y=make_grid(data=data,step=step)
    plt.contourf(x,y,f(grid).reshape(x.shape),colors=('gray','blue'),levels=[-1,0,1])


In [3]:

##################################################################"
class Classifier(object):
    """ Classe generique d'un classifieur
        Dispose de 3 méthodes :
            fit pour apprendre
            predict pour predire
            score pour evaluer la precision
    """
    def fit(self,x,y):
        raise NotImplementedError("fit non  implemente")
    def predict(self,x):
        raise NotImplementedError("predict non implemente")
    def score(self,x,y):
        return (self.predict(x)==y).mean()


In [4]:
def load_usps(filename):
    with open (filename, "r" ) as f :
        f.readline()
        data =[ [ float ( x ) for x in l.split()]for l in f if len(l.split())>2]
    tmp = np.array(data)
    return tmp[:,1:],tmp[:,0].astype(int)
datax,datay = load_usps("train.txt")

In [5]:
# datax , datay = gen_arti(data_type = 0 , nbex = 1000)
# plot_data( datax , datay ) 
# plot_frontiere( datax , f , step=20)

In [6]:
class Bayes(Classifier):
    def fit(self,x,y):
        nbClass = len(np.unique(y))
        xLabel = np.array([x[np.where(y==i)] for i in range(nbClass)])
        mean = []
        std = []
        for j in range(nbClass):
            mean.append([np.mean(xLabel[j][:,i]) for i in range(x.shape[1])])
            std.append([np.var(xLabel[j][:,i])+.01 for i in range(x.shape[1])])
        self.param = np.array(mean) , np.array(std)
        
            
    def predict(self,x):
        m,s = self.param
        
        maxArg =[]
        for k in range(x.shape[0]):
            maxArg.append(np.argmax([np.sum([np.log(1/(np.sqrt(2*np.pi)*s[j][i])*np.exp((-0.5*(float(x[k][i]-m[j][i])/s[j][i])**2))) for i in range(x.shape[1])]) for j in range(m.shape[0])]))
            
        return maxArg
    
    

In [7]:
class Knn(Classifier):
    def fit(self,x,y):
        pass
    def predict(self,x,y):
        pass

In [12]:
b = Bayes()
b.fit(datax,datay)

print b.score(datax,datay)


0.816486078727


/home/melki/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: RuntimeWarning: divide by zero encountered in log


In [15]:
a = np.array(datax)
plot_frontiere(a,b.predict)

/home/melki/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:72: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/home/melki/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: RuntimeWarning: divide by zero encountered in log


AttributeError: 'list' object has no attribute 'reshape'

In [59]:
xLabel = np.array([datax[np.where(datay==i)] for i in range(10)])
xLabel[1].shape

(1005L, 256L)

In [21]:
datax.shape

(7291, 256)

In [173]:
x,y =gen_arti()

In [174]:
x.shape


(1000L, 2L)